In [ ]:
import json
with open("/Tmp/lvpoellhuber/datasets/vault/wikipedia/downloads/AA/wiki_00", "r") as f:
    lines = f.readlines()


In [45]:
import re
import json

BLACKLIST_HEADERS = {
    "References", "Bibliography", "Notes", "External links", "See also",
    "Citations", "Further reading", "Sources"
}

def is_header(line):
    """Heuristic: detect if a line is a section header."""
    if not line:
        return False
    
    if not line[0].isupper():
        return False
    if not line.endswith("."):
        return False
    
    words = line.split()
    if len(words) > 8:
        return False
    
    header_name = line.rstrip(".")
    if header_name in BLACKLIST_HEADERS:
        return False
    
    if len(words) <= 6 and not re.search(r"[!?]", line):
        return True
    
    return False


def chunk_text(text, max_words):
    """Split text into chunks of at most max_words."""
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i:i+max_words])
        chunks.append(chunk)
    return chunks


def split_sections(text, max_words=512):
    lines = [l.strip() for l in text.split("\n") if l.strip()]
    sections = []
    current_header = "Introduction"
    current_text = []

    for line in lines:
        if is_header(line):
            # Save the previous section if it has content
            if current_text:
                full_text = current_header + ". " + " ".join(current_text).strip()
                sections.extend(chunk_text(full_text, max_words))
                current_text = []
            current_header = line.rstrip(".")
        else:
            current_text.append(line)

    # Save the last section
    if current_text:
        full_text = current_header + ". " + " ".join(current_text).strip()
        sections.extend(chunk_text(full_text, max_words))

    return sections

all_sections = []
for line in lines:
    obj = json.loads(line)
    text = obj["text"]
    sections = split_sections(text)
    all_sections.append(sections)

In [48]:
import sqlite3

corpus_path = "/Tmp/lvpoellhuber/datasets/vault/test/corpus.db"
conn = sqlite3.connect(corpus_path)
c = conn.cursor()

# Get first 10 items in corpus
c.execute("SELECT * FROM articles LIMIT 10")
rows = c.fetchall()

In [49]:
rows

[('573199-0',
  'Epic Games',
  'Introduction. American video game company Epic Games, Inc. is an American video game and software developer and publisher based in Cary, North Carolina. The company was founded by Tim Sweeney as Potomac Computer Systems in 1991, originally located in his parents\' house in Potomac, Maryland. Following its first commercial video game release, "ZZT" (1991), the company became Epic MegaGames, Inc. in early 1992 and brought on Mark Rein, who has been its vice president since. After moving the headquarters to Cary in 1999, the studio changed its name to Epic Games. Epic Games developed Unreal Engine, a commercially available game engine which also powers its internally developed video games like "Fortnite" and the "Unreal", "Gears of War", and "Infinity Blade" series. In 2014, Unreal Engine was named the "most successful videogame engine" by "Guinness World Records". Epic Games owns the game developers Psyonix, Mediatonic, and Harmonix, and operates studios 

In [1]:
import faiss
import numpy as np

# 1. Load FAISS index
index = faiss.read_index("/Tmp/lvpoellhuber/models/longtriever_og/hier_ablation_doc-start_sep/default.flat.faiss")

# 2. Inspect number of vectors
print("Index size:", index.ntotal)



Index size: 3201821


In [53]:
id_to_row = {}
with open("/Tmp/lvpoellhuber/models/longtriever_og/hier_ablation_doc-start_sep/default.flat.tsv", "r", encoding="utf-8") as f:
    for row, line in enumerate(f):
        docid, *_ = line.strip().split("\t", 1)
        id_to_row[docid] = row

In [59]:
# Assume id_to_row is already built from default.flat.tsv
row_id = id_to_row["D1660526"]

# Directly reconstruct the vector
vec = index.reconstruct(row_id)  # returns a NumPy array
vec

array([-2.23014280e-01,  6.20490253e-01,  1.66665614e-01, -2.70371940e-02,
        3.64657462e-01,  3.78028214e-01,  7.25720286e-01,  1.31429628e-01,
       -5.36829591e-01, -4.34825748e-01, -2.71385759e-01,  3.65433216e-01,
       -5.71128964e-01,  4.32845235e-01,  8.63932729e-01,  1.78465736e+00,
        9.64794576e-01,  1.24612212e-01,  1.82902634e-01, -9.46981847e-01,
        1.16889805e-01,  2.48504907e-01,  1.79586351e-01, -7.24688265e-03,
        8.63356054e-01,  4.30272520e-01,  3.10706019e-01,  4.80593406e-02,
       -9.47383583e-01,  8.70712996e-02,  2.73059040e-01,  1.89184606e-01,
        6.28192067e-01,  8.24184269e-02, -9.84563768e-01,  1.00192115e-01,
       -6.80273116e-01, -1.15406327e-01,  1.51128978e-01, -1.03684604e+00,
       -2.63928086e-01, -4.67673451e-01, -7.71329582e-01,  4.08610441e-02,
        4.91897106e-01, -3.42191100e-01, -4.43251967e-01,  4.15993661e-01,
       -4.91745144e-01, -4.91936475e-01, -9.77548480e-01,  7.67214954e-01,
        1.57243097e+00,  

In [ ]:
import os
import ir_datasets
from tqdm import tqdm
import json
import re
import pandas as pd

dataset = ir_datasets.load(f"beir/hotpotqa/train")
train_qrels = list(dataset.qrels_iter())
relevant_doc_ids = set(qrel.doc_id for qrel in train_qrels)
download_ir = "/Tmp/lvpoellhuber/datasets/vault/wikipedia/downloads"

total_nb_sentences = 0
nb_sentences = []
nb_words = []
for root, _, files in tqdm(os.walk(download_ir), total=len(os.listdir(download_ir))):
    for filename in files:
        if filename.startswith("wiki_"):
            filepath = os.path.join(root, filename)

            # 3. Read file line-by-line (streaming, not loading whole file in memory)
            with open(filepath, "r", encoding="utf-8") as f:
                for line in f:
                    article = json.loads(line)
                    if article["id"] in relevant_doc_ids:
                        sentences = re.split(r'(?<=[.!?])\s+', article["text"].strip())
                        total_nb_sentences += len(sentences)
                        nb_sentences.append(len(sentences))
                        nb_words.append(len(article["text"].strip().split()))

sentence_df = pd.Series(nb_sentences)
word_df = pd.Series(nb_words)
total_nb_sentences

100%|██████████| 139/139 [00:51<00:00,  2.69it/s]


7313011

In [28]:
sentence_df.describe()

count    88142.000000
mean        82.968517
std        109.059080
min          1.000000
25%         18.000000
50%         43.000000
75%         99.000000
max       1277.000000
dtype: float64

In [31]:
word_df.describe()

count    88142.000000
mean      1632.112126
std       2307.771509
min          5.000000
25%        295.000000
50%        768.000000
75%       1927.000000
max      28332.000000
dtype: float64

In [32]:
word_df[word_df>4096]

0         6600
29        4301
41        8758
52       10715
74       13186
         ...  
88087     8448
88092     4297
88105     5215
88131     4971
88133    13816
Length: 9274, dtype: int64

In [23]:
# avg_vector_storage = 2185439277/3201821 # In bytes
avg_vector_storage = 2.1/3201821 # In GB
avg_vector_storage 

6.558767651283442e-07

In [24]:
estimated_total_storage = total_nb_sentences * avg_vector_storage
estimated_total_storage

4.7964339980279975

In [9]:
import requests
trec_url = "https://ir.nist.gov/covidSubmit/data/"
files = ["qrels-covid_d5_j0.5-5.txt"]
qrel_path = "/Tmp/lvpoellhuber/datasets/vault/cord19/downloads/qrels-covid_d5_j0.5-5.txt"

for file in files:
    file_url = trec_url+file

    with requests.get(file_url, stream=True) as r:
        r.raise_for_status()
        with open(qrel_path, 'w') as f:
            f.write(r.text) 


In [10]:
qrel_path = "/Tmp/lvpoellhuber/datasets/vault/cord19/downloads/qrels-covid_d5_j0.5-5.txt"
relevant_doc_ids = set()
with open(qrel_path, "r", encoding="utf-8") as f:
    for line in f.readlines():
        qid, _, cord_id, score = line.strip().split(" ")
        if int(score) > 0:
            relevant_doc_ids.add(cord_id)

In [16]:
from tqdm import tqdm
import tarfile
import io
import os
import pandas as pd
import json
import re


total_nb_sentences = 0
nb_sentences = []
nb_words = []
cord_dir = "/Tmp/lvpoellhuber/datasets/vault/cord19/downloads"
id_dict = {}
for filename in tqdm(os.listdir(cord_dir)):
    if filename.startswith("cord-19"):
        tar_path = os.path.join(cord_dir, filename)
        date = filename.split("_")[-1].split(".")[0]

        # Open the cord-19 tar archive
        with tarfile.open(tar_path, "r") as outer_tar:
            # Get metadata information, and append it to the ID Table
            metadata_member = outer_tar.getmember(f"{date}/metadata.csv")
            f = outer_tar.extractfile(metadata_member)
            df = pd.read_csv(f)[["sha", "cord_uid"]].dropna()
            id_dict = dict(zip(df["sha"], df["cord_uid"]))
            id_dict.update(id_dict)

            for member in tqdm(outer_tar.getmembers()):                                                
                if member.name.endswith(".tar.gz"):
                    # Extract the inner JSON archives
                    f = outer_tar.extractfile(member)
                    inner_bytes = f.read()
                    inner_fileobj = io.BytesIO(inner_bytes)

                    # Open the inner tar.gz
                    with tarfile.open(fileobj=inner_fileobj, mode="r") as inner_tar:
                        for inner_member in inner_tar.getmembers():
                            if inner_member.name.endswith(".json"):
                                with inner_tar.extractfile(inner_member) as inner_f:
                                    article = json.load(inner_f)
                                    cord_id = id_dict.get(article["paper_id"], None)
                                    if (cord_id is not None) and (cord_id in relevant_doc_ids):
                                        title = article["metadata"]["title"]
                                        abstract = " ".join([text["text"] for text in article["abstract"]])
                                        body_text = " ".join([text["text"] for text in article["body_text"]])
                                        full_text = abstract+" "+body_text
                                        sentences = re.split(r'(?<=[.!?])\s+', full_text.strip())
                                        total_nb_sentences += len(sentences)
                                        nb_sentences.append(len(sentences))
                                        nb_words.append(len(full_text.strip().split()))

sentence_df = pd.Series(nb_sentences)
word_df = pd.Series(nb_words)
total_nb_sentences

  0%|          | 0/7 [00:00<?, ?it/s]/tmp/ipykernel_994925/903382106.py:25: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)[["sha", "cord_uid"]].dropna()
 14%|█▍        | 1/7 [01:19<07:54, 79.04s/it]/tmp/ipykernel_994925/903382106.py:25: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)[["sha", "cord_uid"]].dropna()
 43%|████▎     | 3/7 [04:42<06:23, 95.92s/it]/tmp/ipykernel_994925/903382106.py:25: DtypeWarning: Columns (5,13,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)[["sha", "cord_uid"]].dropna()
 71%|███████▏  | 5/7 [08:49<03:48, 114.18s/it]/tmp/ipykernel_994925/903382106.py:25: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)[["sha", "cord_uid"]].dropna()
100%|██████████| 7/7 [10:33<00:

4777994

In [19]:
full_text

'To examine and summarize the current literature on serologic methods for the detection of antibodies to severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). Methods: A literature review was performed using searches in databases including PubMed, medRxiv, and bioRxiv. Thirty-two peer-reviewed papers and 23 preprints were examined. The studies included lateral flow immunoassay, enzyme-linked immunosorbent assay, chemiluminescence immunoassay, and neutralizing antibody assays. The use of all major SARS-CoV-2 antigens was demonstrated to have diagnostic value. Assays measuring total antibody reactivity had the highest sensitivity. In addition, all the methods provided opportunities to characterize the humoral immune response by isotype. The combined use of IgM and IgG detection resulted in a higher sensitivity than that observed when detecting either isotype alone. Although IgA was rarely studied, it was also demonstrated to be a sensitive marker of infection, and levels correlat

In [21]:
sentence_df.describe()

count    34030.000000
mean       140.405348
std        126.465784
min          1.000000
25%         65.000000
50%        118.000000
75%        179.000000
max       4424.000000
dtype: float64

In [18]:
word_df.describe()

count    34030.000000
mean      3180.758301
std       2922.130850
min          2.000000
25%       1462.000000
50%       2685.000000
75%       4018.000000
max      99960.000000
dtype: float64